<a href="https://colab.research.google.com/github/SarveshPatil99/Adversarial-Robustness-Enhancement/blob/main/noise_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [78]:
noise_params = {
  'gaussian': {'stddev': 0.2},
  'saltpepper': {'pepper_thresh': 0.025,'salt_thresh': 0.025},
  'speckle': {'stddev': 0.2},
  'uniform': {'thresh': 0.2}
}

In [79]:
def add_gaussian_noise(image):
  image = image + np.random.normal(0,noise_params['gaussian']['stddev'],image.shape)
  image = np.clip(image, 0,1)
  return image

def add_saltpepper_noise(image):
  pepper_thresh = noise_params['saltpepper']['pepper_thresh']
  salt_thresh = noise_params['saltpepper']['salt_thresh']

  noise_map = np.random.uniform(0,1,(256,256))
  pepper_map = 1*(noise_map < pepper_thresh)
  pepper_map = np.stack([pepper_map]*3,-1)
  salt_map = 1*(noise_map > (1-salt_thresh))
  salt_map = np.stack([salt_map]*3,-1)

  image = image - 2*pepper_map
  image = image + 2*salt_map
  image = np.clip(image, 0, 1)

  return image

def add_speckle_noise(image):
  image = image + image*np.random.normal(0,noise_params['speckle']['stddev'],image.shape)
  image = np.clip(image, 0,1)
  return image

def add_uniform_noise(image):
  image = image + np.random.uniform(-noise_params['uniform']['thresh'],noise_params['uniform']['thresh'],image.shape)
  image = np.clip(image, 0,1)
  return image

In [ ]:
for noise_func in [add_gaussian_noise, add_saltpepper_noise, add_speckle_noise, add_uniform_noise]:
  plt.figure(figsize=(9,3.5))
  image = cv2.imread('09000.png')/255
  noisy_image = noise_func(image)
  img_list = [image, noisy_image, abs(noisy_image-image)]
  title_list = ['Image', 'Noisy Image', 'Noisy Image - Image']
  for i, img in enumerate(img_list):
    plt.subplot(1,3,i + 1)
    plt.imshow(img[:,:,::-1])
    plt.axis('off')
    plt.title(title_list[i])
  plt.tight_layout()
  plt.savefig(f"{noise_func.__name__.split('_')[1]}.png", bbox_inches='tight')